In [1]:
#import functions
import sys
import glob
import numpy as np


#import local functions
sys.path.append('/home/crombez/Documents/PhD/python/MOPDC/fonction/') 
from data_visualisation import *
from load_data import *
from matrix_tools import *
from walsh_hadamard import *
from walsh_ordered_recon import *

In [2]:
%matplotlib 
#inline 

Using matplotlib backend: Qt5Agg


In [3]:
path_data = "/home/crombez/Documents/PhD/python/MOPDC/Data_save/Recon_data/"
name_data = "rc_clean20_egfp_dsded2_432x512x128.npy"
rc_hyper_cube = np.load(path_data+name_data)
# Hyper cube dimensions
Nl = 432 # number of pixcels along the y dimensions 
Nc = 512 # number of measured Walsh_Hadmard coefficients (correpond to the h dimensions)
Nd = 128 # number of pixcels along the spectral dimensions 

In [24]:
rc_hyper_cube = rc_hyper_cube[:,:,::-1]


In [5]:
Stack = np.zeros((Nl,Nc))
for i in range(Nd):
    Stack += rc_hyper_cube[:,:,i]
plot_im2D(Stack,cmap='gray',title='Stack')

In [6]:
EGFP_path = "/home/crombez/Documents/PhD/python/Spectral_unmix/spectrum/EGFP_fpbase_spectra_500_608.csv"
DsRed2_path = "/home/crombez/Documents/PhD/python/Spectral_unmix/spectrum/DsRed2_fpbase_spectra_500_608.csv"
data_DsRed2 = np.genfromtxt(DsRed2_path, delimiter=',',skip_header=True)
data_EGFP = np.genfromtxt(EGFP_path, delimiter=',',skip_header=True)

In [7]:
L_lambda = [500+i*(108/(Nd-1)) for i in range(Nd)]

In [8]:
L_dim = np.zeros(Nd)
L = data_EGFP[:,1]#data_DsRed2[:,1]#
for i in range(Nd):
    val = L_lambda[i]
    int_val = int(val)
    if val-int_val == 0.0:
        L_dim[i] = L[int_val-500]
    else:
        poid = val%1
        L_dim[i] = (1-poid)*L[int_val-500]+poid*L[int_val-500+1]
simple_plot_2D(L_lambda,L_dim)
L_EGFP = L_dim

In [9]:
L_dim = np.zeros(Nd)
L = data_DsRed2[:,1]#
for i in range(Nd):
    val = L_lambda[i]
    int_val = int(val)
    if val-int_val == 0.0:
        L_dim[i] = L[int_val-500]
    else:
        poid = val%1
        L_dim[i] = (1-poid)*L[int_val-500]+poid*L[int_val-500+1]
simple_plot_2D(L_lambda,L_dim)
L_DsRed2 = L_dim

In [10]:
path_el_fluo = "/home/crombez/Documents/PhD/python/Spectral_unmix/spectrum/spec_auto_fluo.npy"
El_fluo = np.load(path_el_fluo)
simple_plot_2D(L_lambda,El_fluo)

In [11]:
Filt = np.zeros(Nd)
Filt[:21] = np.ones(21)
Filt[57:] = np.ones(128-57)
Filt[21:37] = [ 1*(16-i)/16 for i in range(16)]
Filt[41:57] = [ 1*(i)/16 for i in range(16)]
Filt = smooth(Filt,2)
Filt[:10] = np.ones(10)
Filt[118:] = np.ones(10)
simple_plot_2D([i for i in range(128)],Filt)

# NMF

In [25]:
non_neg = np.where(rc_hyper_cube > 0, rc_hyper_cube, 0)

In [26]:
data = np.zeros((Nl*Nc,Nd))
data_size = Nl*Nc
i = 0
for l in range(Nl):
    for c in range(Nc):
        data[i] = non_neg[l,c]
        i += 1
print(Nl*Nc,i)

221184 221184


In [16]:
stack = np.zeros((Nl,Nc))
for i in range(Nd):
    stack += non_neg[:,:,i]
plot_im2D(stack,cmap='gray')

In [15]:
L_st = 250#50#200#260#150#30#50#
L_ed = 320#400#250#320#200#380#250#
im_area = np.zeros((Nl,Nc))
im_area[L_st:L_ed] = stack[L_st:L_ed]
plot_im2D(im_area,cmap='gray',title='Selected area',xlabel='x',ylabel='y')

In [ ]:
threch = 1800#1100#2200#1300#1800#
im_seuil = np.where(im_area>threch,1,0)
im_control = np.where(im_area>threch,im_area,0)
plot_im2D(im_seuil,cmap='gray',title='Threshold image of the hyper cube stack',xlabel='x',ylabel='y')
plot_im2D(im_control,cmap='gray',title='Control image for the tresholding',xlabel='x',ylabel='y')

In [17]:
data_size = np.sum(im_seuil)
data = np.zeros((data_size,Nd))
i = 0
for l in range(Nl):
    for c in range(Nc):
        if im_seuil[l,c] == 1:
            data[i] = non_neg[l,c]
            i += 1
print(data_size,i)

NameError: name 'im_seuil' is not defined

In [27]:
H1 = np.zeros((6,Nd))
H1[0] = L_DsRed2*Filt#speco/np.max(speco)#_band
H1[1] = L_EGFP*Filt#Dsredo#(np.random.rand(128))/10#_band
H1[2] = El_fluo#(np.random.rand(128))/10#(np.random.rand(128))/10#New_AF2/np.max(New_AF2)*0.5#(np.random.rand(128))/10#(np.random.rand(128))/10#_band
H1[3] = (np.random.rand(128))/10#(np.random.rand(128))/10#(np.random.rand(128))/10#El_fluo#(np.random.rand(128))/10#Test4##fluo#*(1/0.7)#noise#
H1[4] = (np.random.rand(128))/10#
H1[5] = (np.random.rand(128))/10#
W1 = np.dot(data,np.transpose(H1))
np.shape(W1)

(221184, 6)

In [ ]:
H1 = np.zeros((7,Nd))
H1[0] = Htop[0]#L_DsRed2*Filt#speco/np.max(speco)#_band
H1[1] = Htop[1]#L_EGFP*Filt#Dsredo#(np.random.rand(128))/10#_band
H1[2] = Htop[2]#El_fluo#(np.random.rand(128))/10#(np.random.rand(128))/10#New_AF2/np.max(New_AF2)*0.5#(np.random.rand(128))/10#(np.random.rand(128))/10#_band
H1[3] = Htop[3]#(np.random.rand(128))/10#(np.random.rand(128))/10#(np.random.rand(128))/10#El_fluo#(np.random.rand(128))/10#Test4##fluo#*(1/0.7)#noise#
H1[4] = Htop[4]#(np.random.rand(128))/10#
H1[5] = (np.random.rand(128))/10#
H1[6] = (np.random.rand(128))/10#
W1 = np.dot(data,np.transpose(H1))
np.shape(W1)

In [28]:
from sklearn.decomposition import NMF
X = data
comp = 6
H2 = H1[:comp]
W1 = np.dot(data,np.transpose(H2))
model = NMF(n_components=comp, init='custom', random_state=0,max_iter=3500)#'custom'
W = model.fit_transform(X,y=None, W=W1,H=H2)
H = model.components_

In [ ]:
from sklearn.decomposition import NMF
X = data
comp = 5
model = NMF(n_components=comp, init='random', random_state=0,max_iter=3500)#'custom'
W = model.fit_transform(X)#,y=None, W=W1,H=H1)
H = model.components_

In [30]:
L_d = [500+i*(108/(Nd-1)) for i in range(Nd)]
#L_d = [i for i in range(Nd)]
H = H#/np.max(H)
plt.figure(42)
plt.clf()
for i in range(comp):
    plt.plot(L_d,H[i]/np.max(H[i]),label=str(i))#
plt.legend()
plt.ylabel('normalized intensity')
plt.xlabel('wavelenght (nm)')
plt.show()

In [ ]:
1

In [31]:
resto = np.zeros((Nl,Nc,comp))
for ncomp in range(comp):
    i = 0
    for l in range(Nl):
        for c in range(Nc):
            resto[l,c,ncomp] = W[i,ncomp]
            i += 1

In [32]:
for i in range(comp):
    plot_im2D(resto[:,:,i],cmap='gray',title=str(i))

In [ ]:
for i in range(comp):
    plot_im2D(np.log(resto[:,:,i]+10),cmap='gray',title=str(i))

In [ ]:
Htop =H

# Abondance Map

In [ ]:
import pysptools as pysptools
from pysptools import noise,abundance_maps

In [ ]:
comp = 3
H = np.zeros((comp,Nd))
H[0] = L_DsRed2*Filt
H[1] = L_EGFP*Filt
H[2] = El_fluo#New_AF/np.max(New_AF)*0.5#AF#
#H[3] = np.random.rand(128)/10#Test5
#H[4] = np.random.rand(128)/10#NOISE/np.max(NOISE)#np.ones(Nd)*0.052#np.random.rand(128)/10#oise#(
U = H
M = data

In [ ]:
U = H
M = data

In [ ]:
plt.figure(1)

plt.clf()
plt.plot(L_lambda,H[0],'r',label = 'DsRed2')
plt.plot(L_lambda, H[1], 'g', label = 'EGFP')#plt.plot(L_lambda, H[1]/np.max(H[1]),'b', label = 'AF')#
plt.plot(L_lambda, H[2],'b', label = 'AF')#
#plt.plot(L_lambda, H[3],'c', label = 'noise')
#plt.plot(L_lambda, H[4],'k', label = 'Sig?')
plt.legend()
plt.xlabel('Wavelenght (nm)')
plt.ylabel('Intensitiy (normalized)')
plt.show()

In [ ]:
Abondance = abundance_maps.amaps.UCLS(M, U)

In [ ]:
%%time
Abondance = abundance_maps.amaps.NNLS(M, U)

In [ ]:
Abondance = abundance_maps.amaps.FCLS(M, U)

In [ ]:
Abond = np.zeros((Nl,Nc,comp))
for ncomp in range(comp):
    i = 0
    for l in range(Nl):
        for c in range(Nc):
            Abond [l,c,ncomp] = Abondance[i,ncomp]
            i += 1

In [ ]:
for i in range(comp): 
    plot_im2D(Abond[:,:,i],cmap='gray',title=str(i))

In [ ]:
for i in range(comp):
    plot_im2D(np.log(Abond[:,:,i]+10),cmap='gray',title=str(i))

In [ ]:
ab_stack = np.zeros((Nl,Nc))
for i in range(comp):
    ab_stack += Abond[:,:,i]*np.sum(H[i])
plot_im2D(Stack-ab_stack)

# PCA

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
X = data #- np.mean(data,0)
comp = 8
pca = PCA(n_components=comp)
pca.fit(X)

In [ ]:
V_pca = np.transpose(pca.components_)
rc_hyp_pca = np.dot(rc_hyper_cube,V_pca)

In [ ]:
L_d = [i for i in range(Nd)]
for c_pca in range(comp):
    simple_plot_2D(L_d,V_pca[:,c_pca],title='Componant '+str(c_pca),xlabel='Spectral channel',ylabel='Intensity')

In [ ]:
for i in range(comp):
    plot_im2D(rc_hyp_pca[:,:,i],title='Componant '+str(i))

In [ ]:

a = 0
b = -1
c = 1
d = -1
e = 0

im =a*rc_hyp_pca[:,:,0]+b*rc_hyp_pca[:,:,1]+c*rc_hyp_pca[:,:,2]+d*rc_hyp_pca[:,:,3]+e*rc_hyp_pca[:,:,4]

plot_im2D(im)

In [ ]:
im_comp = rc_hyp_pca[:,:,1]
plot_im2D(np.where(im_comp>0,im_comp,0))